In [2]:
import feature_engineering as fe
import train_utils as tu
from persistent_cache import memo, PersistentDict as Perd
from sklearn.cross_validation import StratifiedKFold, KFold
import numpy as np

In [ ]:
X_train, X_test = fe.fe2()
y = fe.get_y()

In [ ]:
X_train.shape

In [ ]:
train_1, test_1 = fe.add_poly_feats(X_train, X_test, y, 0.2)

In [ ]:
train_1.shape

In [3]:
tu.lazy_benchmark(tu.xgbr(), "feats2")

0.5967406703428549

In [3]:
tu.lazy_benchmark(tu.xgbr(), "fe2_median")

0.5930379591214243

In [2]:
tu.lazy_benchmark(tu.xgbr(), "fe2_poly02")

0.5975717241145663

In [2]:
tu.lazy_benchmark(tu.xgbr(), "fe2_poly02")

0.5980794201664619

In [5]:
tu.lazy_benchmark(tu.xgbr(), "fe2_poly01")

0.5951555037475283

In [3]:
tu.lazy_benchmark(tu.xgbr(), "fe2_poly01")

0.5959702161748921

In [6]:
tu.lazy_benchmark(tu.xgbr(), "fe2_poly015")

0.5954136714888252

In [4]:
tu.lazy_benchmark(tu.xgbr(), "fe2_poly015")

0.5980583065350319

In [3]:
tu.lazy_benchmark(tu.xgbr(), "fe2_polyomc02")

0.5975439790898074